In [ ]:
pip install lightgbm xgboost

In [46]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from sklearn.feature_selection import RFECV, RFE
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC

pd.set_option('display.max_columns', None, 'display.max_rows', None)

In [47]:
## Reading the data
train = pd.read_csv('Data/W23P2_train.csv')
test = pd.read_csv('Data/W23P2_test.csv')
sub = pd.read_csv('Data/Sample_Submission.csv')

In [48]:
train.head()

,interest_level,bathrooms,bedrooms,latitude,longitude,price,street_address,Dining.Room,Pre.War,Laundry.in.Building,Dishwasher,Hardwood.Floors,Dogs.Allowed,Cats.Allowed,Doorman,Elevator,No.Fee,Laundry.in.Unit,Swimming.Pool,Roof.Deck,Fitness.Center,High.Speed.Internet,Wheelchair.Access,Reduced.Fee,Common.Outdoor.Space,Parking.Space,Laundry.In.Building,Concierge,PublicOutdoor,Garage,Laundry.In.Unit,New.Construction,Exclusive,Private.Outdoor.Space,Garden.Patio,Outdoor.Space,Common.roof.deck,Fireplace,Storage,dishwasher,Newly.renovated,No.pets,prewar,LOWRISE,LAUNDRY,SIMPLEX,HARDWOOD,Balcony,Furnished,Terrace,High.ceilings,Loft,High.Ceilings,Hardwood.floors,Stainless.Steel.Appliances,Central.A.C,Multi.Level,Garden,View,Concierge.Service,Prewar,Patio,Roof.deck,Laundry.Room,elevator,pool,Full.time.doorman,Gym.Fitness,Pool,LIVE.IN.SUPER,HIGH.CEILINGS,Washer.in.Unit,Dryer.in.Unit,Green.Building,Short.Term.Allowed,Subway,Laundry,Hardwood,Renovated,Marble.Bath,Granite.Kitchen,Light,Pet.Friendly,Guarantors.Accepted,storage,EXPOSED.BRICK,Air.conditioning,Live.in.superintendent,On.site.laundry,High.Ceiling,WiFi,Lounge,Actual.Apt..Photos,Skylight,Parking,EAT.IN.KITCHEN,Sauna,Live.Work,Duplex,Pets.on.approval,NO.FEE,Eat.In.Kitchen,ACTUAL.APT..PHOTOS,X24.7.Concierge,Virtual.Doorman,On.site.Garage,Valet,renovated,New.construction,Bike.room,HIGHRISE,ROOFDECK,private.outdoor.space,balcony,Walk.in.Closet.s.,On.site.Laundry,assigned.parking.space,Private.parking,Washer.Dryer,Luxury.building,Post.War,Indoor.Pool,Residents.Lounge,WiFi.Access,Valet.Parking,Common.parking.Garage,Courtyard,Lounge.room,Childrens.Playroom,Residents.Garden,Marble.Bathroom,FT.Doorman,Hi.Rise,Basement.Storage,Full.Service.Garage,Health.Club,Washer.Dryer.in.building,Children.s.Playroom,Outdoor.Entertainment.Space,In.Unit.Washer.Dryer,Bike.Room,Live.In.Superintendent,Live.in.Super,doorman,On.site.Parking.Lot,Cable.Satellite.TV,Microwave,Outdoor.Areas,Decorative.Fireplace,terrace,Gym,Sublet,Live.In.Super,Wheelchair.Ramp,private.balcony,Private.balcony,Common.backyard,Washer...Dryer,Multi.level,patio,Exposed.Brick,building.common.outdoor.space,garden,Washer.Dryer.in.Unit,Private.Deck,Shares.OK,gym,Private.outdoor.space,Gym.In.Building,Common.garden
0,medium,1.0,4,40.7831,-73.9449,3675,1962 First Avenue,0,0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,medium,1.0,3,40.7542,-73.9724,3600,155 East 47th Street,1,0,1,1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,medium,1.0,2,40.7285,-73.9790,2900,206 Avenue B,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,medium,2.0,4,40.7306,-73.9837,7200,352 E 13 St,1,1,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,high,1.0,1,40.7645,-73.9840,2500,260 W 54 St.,0,0,1,1,1,1,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [49]:
print(train.shape)
print(test.shape)

(9846, 170)
(6564, 170)


In [50]:
train['interest_level'].value_counts()

low       3884
medium    3716
high      2246
Name: interest_level, dtype: int64

In [51]:
## Defining the input and target variables
X_train = train.drop(columns = ['interest_level', 'street_address'])
Y_train = train['interest_level']

X_test = test.drop(columns = ['street_address', 'ID'])

In [40]:
## Building the multi-classifier (using RF)
one_vs_all_RF = OneVsRestClassifier(estimator = RandomForestClassifier(n_estimators = 500, max_depth = 3)).fit(X_train, Y_train)

## Predicting on the test
train_preds = one_vs_all_RF.predict_proba(X_train)
test_preds = one_vs_all_RF.predict_proba(X_test)

## Predictions to data-frames
train_preds = pd.DataFrame(train_preds, columns = ['high', 'low', 'medium'])
test_preds = pd.DataFrame(test_preds, columns = ['high', 'low', 'medium'])

## Printing performance on the training set
train_error = log_loss(Y_train, train_preds)
print(train_error)

## Configuring submission
sub[['high', 'medium', 'low']] = test_preds[['high', 'medium', 'low']]
sub.to_csv('Submissions/Submission1.csv', index = False)

In [52]:
## Building the multi-classifier (using SVM)
one_vs_all_svm = OneVsRestClassifier(estimator = LGBMClassifier()).fit(X_train, Y_train)

## Predicting on the test
train_preds = one_vs_all_svm.predict_proba(X_train)
test_preds = one_vs_all_svm.predict_proba(X_test)

## Predictions to data-frames
train_preds = pd.DataFrame(train_preds, columns = ['high', 'low', 'medium'])
test_preds = pd.DataFrame(test_preds, columns = ['high', 'low', 'medium'])

## Printing performance on the training set
train_error = log_loss(Y_train, train_preds)
print(train_error)

## Configuring submission
sub[['high', 'medium', 'low']] = test_preds[['high', 'medium', 'low']]
sub.to_csv('Submissions/Submission2.csv', index = False)

0.6387222937678705
